In [1]:
#redirecionamento para o torch é importante para instalar bibliotecas de
#otimização necessárias para parte do treino da LLM
!pip install transformers[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
#texto que será usado para treinamento
!wget -O ./sample_data/jcs.txt https://raw.githubusercontent.com/graciellenjose/llm-estudos/refs/heads/main/jcs.txt

--2025-03-14 19:12:13--  https://raw.githubusercontent.com/graciellenjose/llm-estudos/refs/heads/main/jcs.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36624 (36K) [text/plain]
Saving to: ‘./sample_data/jcs.txt’

./sample_data/jcs.t 100%[===================>]  35.77K  --.-KB/s    in 0.002s  

2025-03-14 19:12:14 (22.6 MB/s) - ‘./sample_data/jcs.txt’ saved [36624/36624]



In [3]:
#definindo de onde virá o texto
from os.path import sameopenfile
PATH = './sample_data/'
texto_treino = 'jcs.txt'

In [4]:
#byte level conta as frequências de pares de tokens
#e soma os ids
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

#files - arquivo(s) de treino
#vocab_size - quantidade de espaço para armazenamento de possibilidades
#min_frequency - número de ocorrências dos pares
#special_tokens - caracteres especiais
tokenizer.train(files=[PATH+texto_treino], vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>", #começo da linha
    "<pad>", #preenchimento de espaços em brancos
    "</s>", #final de frase
    "<unk>", #caractere desconhecido
    "<mask>", #caractere para predição
])

In [5]:
tokenizer.encode('Uma frase é um enunciado falado ou escrito que apresenta um sentido completo.').ids

[57,
 81,
 69,
 616,
 367,
 73,
 225,
 132,
 107,
 225,
 89,
 81,
 225,
 310,
 548,
 71,
 77,
 377,
 83,
 285,
 385,
 377,
 83,
 225,
 263,
 225,
 282,
 71,
 86,
 298,
 83,
 853,
 73,
 266,
 84,
 274,
 87,
 503,
 69,
 225,
 89,
 81,
 272,
 503,
 368,
 83,
 712,
 84,
 80,
 315,
 83,
 18]

In [6]:
#salvar tokenização
!rm -r ./sample_data/RAW_MODEL
!mkdir ./sample_data/RAW_MODEL
tokenizer.save_model(PATH+'RAW_MODEL')

rm: cannot remove './sample_data/RAW_MODEL': No such file or directory


['./sample_data/RAW_MODEL/vocab.json', './sample_data/RAW_MODEL/merges.txt']

In [7]:
#modelo de tokenizer BERT
#separar parte de treino da parte de processamento

#from tokenizers.implementations import ByteLevelBPETokenizer
#from tokenizers.processors import BertProcessing

#tokenizer = ByteLevelBPETokenizer(
#    PATH+'RAW_MODEL'+"/vocab.json",
#    PATH+'RAW_MODEL'+"/merges.txt",
#)

#tokenizer._tokenizer.post_processor = BertProcessing(
    #identifica que o <s> e </s> do tokenizer é o mesmo do vocabulário.
    #serve para deiferenciar frases que já foram processadas de frases que não
    #foram
#    ("</s>", tokenizer.token_to_id("</s>")),
#    ("<s>", tokenizer.token_to_id("<s>"))
#)

#tokenizer.enable_truncation(max_length=512)

In [8]:
#o modelo RoBERTa é uma melhoria do BERT que permite que a utilização em
#dados reais seja mais rápida. É otimizada para menores quantidades de dados e
#máquinas com menos recursos
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained(PATH+'RAW_MODEL', max_length=512)
#é pre_trained, por o vocabulário com os merges já existe

In [9]:
#Transformer
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000, #quantidade do vocabulário
    max_position_embeddings=512, #quantidade de embeddings do vetor
    num_attention_heads=12, #garantem que o contexto seja levado em conta
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [12]:
!echo ".ignoreme" > .gitignore
from google.colab import userdata

username = userdata.get('username')
useremail = userdata.get('useremail')
password = userdata.get('password')
token = userdata.get('token')

!git config --global user.name {username}
!git config --global user.email {useremail}
!git config --global user.password {password}

In [ ]:
!echo ".ignoreme" > .gitignore
from google.colab import drive
drive.mount('/content/drive/')

In [14]:
!echo ".ignoreme" > .gitignore
%cd /content/drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [19]:
!echo ".ignoreme" > .gitignore
!git init
!git add basico-llm.ipynb
!git branch -M main
!git commit -m "dados - tokenização - início de transformer"
!git remote add origin https://{token}@github.com/{username}/llm-estudos.txt
!git push -u origin main

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/Colab Notebooks/.git/
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@8d850278189f.(none)')
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/{